In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense


In [2]:
data = pd.read_csv('Dataset/question.csv')
data.head(5)

,question,reponse
0,Qu’est-ce que la transition numérique éco-resp...,Une belle excuse pour vendre du matériel 'nouv...
1,Comment réduire la consommation énergétique de...,"Éteignez-les. Radical, efficace, et vous verre..."
2,Faut-il migrer vers le cloud vert ?,"Oui, car 'cloud vert' c'est comme 'chocolat sa..."
3,Peut-on recycler nos vieux ordinateurs ?,Bien sûr ! Envoyez-les dans un pays en dévelop...
4,Les IA consomment beaucoup d'énergie : que fai...,Demandez-leur gentiment de penser moins fort. ...


In [3]:
questions = data['question'].astype(str).tolist()
answers = data['reponse'].astype(str).tolist()

# Ajouter les tokens de début et fin
answers = ['<start> ' + ans.strip() + ' <end>' for ans in answers]

num_words = 5000  # taille du vocabulaire

In [4]:

tokenizer_questions = Tokenizer(num_words=num_words, oov_token='<OOV>')
tokenizer_questions.fit_on_texts(questions)
questions_seq = tokenizer_questions.texts_to_sequences(questions)
max_len_questions = max([len(seq) for seq in questions_seq])
questions_seq = pad_sequences(questions_seq, maxlen=max_len_questions, padding='post')

# Tokenizer pour les réponses
tokenizer_answers = Tokenizer(num_words=num_words, oov_token='<OOV>', filters='', lower=True)
tokenizer_answers.fit_on_texts(answers)
answers_seq = tokenizer_answers.texts_to_sequences(answers)
max_len_answers = max([len(seq) for seq in answers_seq])
answers_seq = pad_sequences(answers_seq, maxlen=max_len_answers, padding='post')
vocab_size_answers = len(tokenizer_answers.word_index) + 1


In [5]:

#preparation des donnees
decoder_input_seq = answers_seq[:, :-1]
decoder_target_seq = answers_seq[:, 1:]
decoder_target_seq = np.expand_dims(decoder_target_seq, -1)  # pour sparse_categorical_crossentropy


In [6]:


embedding_dim = 128
lstm_units = 256

# Encodeur
encoder_inputs = Input(shape=(max_len_questions,))
enc_emb = Embedding(input_dim=num_words, output_dim=embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Décodeur
decoder_inputs = Input(shape=(max_len_answers-1,))
dec_emb_layer = Embedding(input_dim=vocab_size_answers, output_dim=embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_answers, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# modele
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 11)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 28)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 11, 128)           │         640,000 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, 28, 128)           │         121,600 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 256), (None,      │         394,240 │ embedding[0][0]            │
│                               │ 256), (None, 256)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, 28, 256), (None,  │         394,240 │ embedding_1[0][0],         │
│                               │ 256), (None, 256)]        │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 28, 950)           │         244,150 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,794,230 (6.84 MB)

 Trainable params: 1,794,230 (6.84 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:


#  Entraîner le modèle
model.fit([questions_seq, decoder_input_seq], decoder_target_seq,
          batch_size=16,
          epochs=100,
          validation_split=0.1)

# Créer les modèles d'inférence

# Encodeur pour l'inférence
encoder_model = Model(encoder_inputs, encoder_states)

# Décodeur pour l'inférence
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)






Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 202ms/step - accuracy: 0.2615 - loss: 6.7963 - val_accuracy: 0.3052 - val_loss: 6.5946
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.3147 - loss: 5.6575 - val_accuracy: 0.3019 - val_loss: 5.0810
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.3147 - loss: 4.8734 - val_accuracy: 0.3019 - val_loss: 4.9269
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.3147 - loss: 4.6221 - val_accuracy: 0.3019 - val_loss: 4.9731
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.3151 - loss: 4.4860 - val_accuracy: 0.3052 - val_loss: 4.9891
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.3238 - loss: 4.3450 - val_accuracy: 0.3019 - val_loss: 4.8403
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.3242 - loss: 4.2110 - val_accuracy: 0.3019 - val_loss: 4.8317
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.3270 - loss: 4.1172 - val_accuracy: 0.3084 - val

In [8]:
def generate_response(input_text):
    # Tokenisation et padding
    seq = tokenizer_questions.texts_to_sequences([input_text])
    seq = pad_sequences(seq, maxlen=max_len_questions, padding='post')

    states_value = encoder_model.predict(seq)

    # Initialiser le token de départ
    target_seq = np.array([[tokenizer_answers.word_index['<start>']]])

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Choisir le mot avec la plus haute probabilité
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_answers.index_word.get(sampled_token_index, '')

        if sampled_word == '<end>' or len(decoded_sentence.split()) > max_len_answers:
            stop_condition = True
        else:
            decoded_sentence += sampled_word + ' '

        # Mettre à jour le token et les états
        target_seq = np.array([[sampled_token_index]])
        states_value = [h, c]

    return decoded_sentence.strip()

In [11]:
phrase = "tu as quel age "
response = generate_response(phrase)
print("Question :", phrase)
print("Réponse :", response)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Question : tu as quel age 
Réponse : oui, en plantant des arbres qui mettront 50 ans à absorber ce de émettent en 1 jour. !


In [10]:
## Enregistrement du modele 

model.save("./models/model.keras")